importing dependencies


In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data collection and data pre processing

In [8]:

try:
    movie_data = pd.read_csv('/content/movies.csv', nrows=3200)
    print("Successfully read the first 3200 rows. The issue is likely after this point.")
except pd.errors.ParserError as e:
    print(f"Error still occurs: {e}")
    # If the error still occurs, try narrowing down the problematic rows further.

# If the first 3200 rows read successfully, try reading in chunks:
chunksize = 1000
for chunk in pd.read_csv('/content/movies.csv', chunksize=chunksize):
    try:
        # Process each chunk here
        print("Processed a chunk.")
    except pd.errors.ParserError as e:
        print(f"Error encountered in a chunk: {e}")

Successfully read the first 3200 rows. The issue is likely after this point.
Processed a chunk.
Processed a chunk.
Processed a chunk.
Processed a chunk.
Processed a chunk.


In [10]:
#printing the first 5 rows of the dataframe
movie_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [11]:
#printing the shape
movie_data.shape

(3200, 24)

In [12]:
#selecting the relevant features for recommendation
selected_features = ['genres','keywords','taglines','cast','directors']
print(selected_features)

['genres', 'keywords', 'taglines', 'cast', 'directors']


In [13]:
#replacing the null values with null strings
for feature in selected_features:
  if feature in movie_data.columns:  # Check if column exists before processing
    movie_data[feature] = movie_data[feature].fillna('')
  else:
    print(f"Column '{feature}' not found in the DataFrame.")

Column 'taglines' not found in the DataFrame.
Column 'directors' not found in the DataFrame.


In [17]:
#combined all the 5 features
combined_features = movie_data['genres'].astype(str)
if 'keywords' in movie_data.columns:
    combined_features += ' ' + movie_data['keywords'].astype(str)
if 'taglines' in movie_data.columns:
    combined_features += ' ' + movie_data['taglines'].astype(str)
if 'cast' in movie_data.columns:
    combined_features += ' ' + movie_data['cast'].astype(str)
if 'directors' in movie_data.columns:
    combined_features += ' ' + movie_data['directors'].astype(str)




In [18]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
3195    Drama biography historical figure princess dra...
3196    Drama Family Foreign miner australia terminal ...
3197    Crime Drama History War assassin copenhagen re...
3198    Comedy Romance independent film aspiring singe...
3199    Thriller Mystery Crime Drama  Andrew Garfield ...
Name: genres, Length: 3200, dtype: object


In [15]:
#combining the test data to a feature vectors
vectorizer = TfidfVectorizer()

In [19]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [20]:
print(feature_vectors)

  (0, 7229)	0.23478901506033298
  (0, 5666)	0.1750124419703124
  (0, 4830)	0.260090517695797
  (0, 8099)	0.18981287222395632
  (0, 9225)	0.21219262773853378
  (0, 7806)	0.21829049480329724
  (0, 7406)	0.23227061295366183
  (0, 9561)	0.22364186647826736
  (0, 9406)	0.2509169498767044
  (0, 7424)	0.1740123149596686
  (0, 7919)	0.23749413037399777
  (0, 1744)	0.2723725458427698
  (0, 9185)	0.1382714514573016
  (0, 7976)	0.3710270287042649
  (0, 3162)	0.19834036384280335
  (0, 1652)	0.26571738013034835
  (0, 1994)	0.26571738013034835
  (0, 2957)	0.11081154562664429
  (0, 7558)	0.11081154562664429
  (0, 2871)	0.11516552321893804
  (0, 95)	0.09193886379474606
  (0, 57)	0.08057584559839896
  (1, 2890)	0.21952672679223376
  (1, 9505)	0.21666438639308286
  (1, 1598)	0.21159342057583438
  :	:
  (3198, 4662)	0.27186086774430734
  (3198, 4082)	0.17803524106036683
  (3198, 8108)	0.20095321670455654
  (3198, 1378)	0.20550655335831153
  (3198, 294)	0.20240447283612176
  (3198, 5118)	0.202404472836121

Cosine Similar

In [21]:
#getting the similarity scores using cosins
similarity = cosine_similarity(feature_vectors)

In [22]:
print(similarity)

[[1.         0.02542152 0.01606601 ... 0.03397099 0.         0.        ]
 [0.02542152 1.         0.01447879 ... 0.         0.         0.        ]
 [0.01606601 0.01447879 1.         ... 0.01026887 0.         0.01550745]
 ...
 [0.03397099 0.         0.01026887 ... 1.         0.         0.01796659]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.01550745 ... 0.01796659 0.         1.        ]]


In [23]:
print(similarity.shape)

(3200, 3200)


In [33]:
#getting the movie name from the user
movie_name = input ('Enter your favourite movie name:')

Enter your favourite movie name:Spectre


In [25]:
#creating the list with all movies name given by user
list_of_all_titles  = movie_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [34]:
# finding the close match the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
close_match = find_close_match[0] # Assign the closest match to close_match
print(find_close_match)

['Spectre', 'Sphere', 'Species']


In [37]:
close_match = find_close_match[0]
print(close_match)

Spectre


In [39]:
#finding the index  of the movie with title
index_of_the_movie = movie_data[movie_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

2


In [40]:
#getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.016066008137386324), (1, 0.01447879329412074), (2, 1.0), (3, 0.06813129633328463), (4, 0.07556394793722916), (5, 0.016500002345830423), (6, 0.0), (7, 0.09411821536442799), (8, 0.04675403934391127), (9, 0.05272819430662872), (10, 0.05773886206865213), (11, 0.2063592260698586), (12, 0.01659997847162928), (13, 0.01492291288685495), (14, 0.05420789488520618), (15, 0.06228049771571127), (16, 0.05316928742036173), (17, 0.016526827652689237), (18, 0.006618691092330769), (19, 0.016526662641828425), (20, 0.016262535780915476), (21, 0.015869897754309532), (22, 0.009832394134728847), (23, 0.09486908865660829), (24, 0.014649904332093601), (25, 0.0), (26, 0.06082747652912229), (27, 0.014792860163441258), (28, 0.016447163325414533), (29, 0.31340277067609756), (30, 0.05520764550708646), (31, 0.03509010505408712), (32, 0.06553620615615709), (33, 0.05153201322642097), (34, 0.0), (35, 0.016860127575194547), (36, 0.05257243353984657), (37, 0.010192580119818443), (38, 0.09151701656664785), (39, 0.0

In [41]:
len(similarity_score)

3200

In [42]:
#sorting the movies based  on their similarity score
sorted_similar_movie = sorted(similarity_score,key = lambda x:x[1], reverse=True)
print(sorted_similar_movie)

[(2, 1.0), (29, 0.31340277067609756), (11, 0.2063592260698586), (103, 0.16990657281197868), (529, 0.1686324712633191), (497, 0.16518952091794364), (252, 0.16459207836367745), (354, 0.16159947031155686), (3047, 0.14970499505459856), (287, 0.1486472793359331), (62, 0.1480491803674191), (571, 0.14155213270873915), (1305, 0.14000802015285876), (113, 0.13906116740017752), (339, 0.13856994308792622), (219, 0.13658687278064016), (1940, 0.1321136364715997), (114, 0.1301577575374808), (1769, 0.1282576986223197), (433, 0.1276020161376895), (1810, 0.1269215820464407), (155, 0.12526519205765244), (1100, 0.12231510165453867), (183, 0.1219052923307636), (147, 0.11908856996501173), (3085, 0.11777051093258098), (1192, 0.11262762772395965), (1137, 0.1106613982691699), (1474, 0.10989151503933489), (134, 0.1097604446516708), (249, 0.10807076850999667), (2006, 0.10580312445114864), (256, 0.1052880805645263), (698, 0.104875407174621), (2169, 0.10406565672299527), (277, 0.10389039920534386), (694, 0.1024345

In [44]:
#print the name of the movies based on the index
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movie:
  index = movie[0]
  title_from_index = movie_data[movie_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Spectre
2 . Skyfall
3 . Quantum of Solace
4 . The Sorcerer's Apprentice
5 . Tears of the Sun
6 . Red Dragon
7 . The Green Hornet
8 . The Girl with the Dragon Tattoo
9 . Big Eyes
10 . Django Unchained
11 . The Legend of Tarzan
12 . Inglourious Basterds
13 . Water for Elephants
14 . Harry Potter and the Order of the Phoenix
15 . The Incredibles
16 . Charlie's Angels: Full Throttle
17 . Carnage
18 . Harry Potter and the Goblet of Fire
19 . Spy Kids: All the Time in the World
20 . RED 2
21 . The Passion of the Christ
22 . Fun with Dick and Jane
23 . Johnny English Reborn
24 . The Hunger Games: Catching Fire
25 . Die Another Day
26 . Spider
27 . Spawn
28 . One for the Money
29 . I Dreamed of Africa


Movie Reccomendation System

In [4]:
movie_name = input ('Enter your favourite movie name:')

list_of_all_title  = movie_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movie_data[movie_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movie = sorted(similarity_score,key = lambda x:x[1], reverse=True)

print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movie:
  index = movie[0]
  title_from_index = movie_data[movie_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1



NameError: name 'movie_data' is not defined